# Goal: to build a model that predicts if a weapon was used or not based on the attributes of that crime

This will use a binary outcome of true or false. I will try using bayesian model, logistic regression, random forrest.

In [1]:
import pandas as pd
import pyspark
import numpy
from pyspark.sql import functions as F

In [2]:
df = pd.read_csv('clean_crime_data.csv')

I am not sure how many crime involved the use of a weapon. Lets look:

In [3]:
len(df[df.firearm_used_flag >=1]) # this feels like a cumbersome approach. Lets do something cool

401

In [4]:
spark = pyspark.sql.SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

# create spark dataframes
crime_df = spark.createDataFrame(df)

crime_df.createOrReplaceTempView('crime')

print('crimes where weapon was used')

gun_crimes = spark.sql("""
select 
    count(distinct crime_id) as crime_count,
    description
from crime
where firearm_used_flag >= 1
and description not LIKE '%Weapons%'
group by 2 order by 1 desc
""")

gun_crimes.show()

print('all crimes')

all_crimes = spark.sql("""
select
count(distinct crime_id) as crime_count
from crime
""")

all_crimes.show()

#print('Weapons were used in {}% of the crimes in this data set')

crimes where weapon was used
+-----------+--------------------+
|crime_count|         description|
+-----------+--------------------+
|         93|  Aggravated Assault|
|         55|Aggravated Assaul...|
|          8|Non Aggravated As...|
|          6|       Armed Robbery|
|          5|Non Aggravated As...|
|          2|  Strong Arm Robbery|
|          1|                Rape|
|          1|            Homocide|
|          1|Kidnapping/Abduction|
+-----------+--------------------+

all crimes
+-----------+
|crime_count|
+-----------+
|      30400|
+-----------+



In [5]:
print('Weapons were used in {}% of the crimes in this data set'.format(round((gun_crimes.groupBy().sum().collect()[0][0]/
                                                                            all_crimes.groupBy().sum().collect()[0][0]),3)*100))

Weapons were used in 0.6% of the crimes in this data set


Ok. This might seem pretty bad but actually there are a ton of types of crimes that we can exclude to narrow our focus and give this percentage a bit more of a fighting chance! 